## complete implementation 1

In [43]:
#-*-coding:UTF-8-*-
'''
Created on 2014年9月25日
@author: Ayumi Phoenix
'''
import numpy as np

class HMM:
    def __init__(self, Ann, Bnm, pi1n):
        self.A = np.array(Ann)
        self.B = np.array(Bnm)
        self.pi = np.array(pi1n)
        self.N = self.A.shape[0]
        self.M = self.B.shape[1]
        
    def printhmm(self):
        print ("==================================================")
        print ("HMM content: N =",self.N,",M =",self.M)
        for i in range(self.N):
            if i==0:
                print ("hmm.A ",self.A[i,:]," hmm.B ",self.B[i,:])
            else:
                print ("      ",self.A[i,:],"       ",self.B[i,:])
        print ("hmm.pi",self.pi)
        print ("==================================================")

    # 函数名称：Forward *功能：前向算法估计参数 *参数:phmm:指向HMM的指针
    # T:观察值序列的长度 O:观察值序列    
    # alpha:运算中用到的临时数组 pprob:返回值,所要求的概率
    def Forward(self,T,O,alpha,pprob):
    #     1. Initialization 初始化
        for i in range(self.N):
            alpha[0,i] = self.pi[i]*self.B[i,O[0]]
    
    #     2. Induction 递归
        for t in range(T-1):
            for j in range(self.N):
                sum = 0.0
                for i in range(self.N):
                    sum += alpha[t,i]*self.A[i,j]
                alpha[t+1,j] =sum*self.B[j,O[t+1]]
    #     3. Termination 终止
        sum = 0.0
        for i in range(self.N):
            sum += alpha[T-1,i]
        pprob[0] *= sum
    
    # 带修正的前向算法
    def ForwardWithScale(self,T,O,alpha,scale,pprob):
        scale[0] = 0.0
    #     1. Initialization
        for i in range(self.N):
            alpha[0,i] = self.pi[i]*self.B[i,O[0]]
            scale[0] += alpha[0,i]
        
        for i in range(self.N):
            alpha[0,i] /= scale[0]
        
    #     2. Induction
        for t in range(T-1):
            scale[t+1] = 0.0
            for j in range(self.N):
                sum = 0.0
                for i in range(self.N):
                    sum += alpha[t,i]*self.A[i,j]
                
                alpha[t+1,j] = sum * self.B[j,O[t+1]]
                scale[t+1] += alpha[t+1,j]
            for j in range(self.N):
                alpha[t+1,j] /= scale[t+1]
        
    #     3. Termination
        for t in range(T):
            pprob[0] += np.log(scale[t])

    # 函数名称：Backward * 功能:后向算法估计参数 * 参数:phmm:指向HMM的指针 
    # T:观察值序列的长度 O:观察值序列 
    # beta:运算中用到的临时数组 pprob:返回值，所要求的概率
    def Backword(self,T,O,beta,pprob):
    #     1. Intialization
        for i in range(self.N):
            beta[T-1,i] = 1.0
    #     2. Induction
        for t in range(T-2,-1,-1):
            for i in range(self.N):
                sum = 0.0
                for j in range(self.N):
                    sum += self.A[i,j]*self.B[j,O[t+1]]*beta[t+1,j]
                beta[t,i] = sum
                
    #     3. Termination
        pprob[0] = 0.0
        for i in range(self.N):
            pprob[0] += self.pi[i]*self.B[i,O[0]]*beta[0,i]
    
    # 带修正的后向算法
    def BackwardWithScale(self,T,O,beta,scale):
    #     1. Intialization
        for i in range(self.N):
            beta[T-1,i] = 1.0
    
    #     2. Induction
        for t in range(T-2,-1,-1):
            for i in range(self.N):
                sum = 0.0
                for j in range(self.N):
                    sum += self.A[i,j]*self.B[j,O[t+1]]*beta[t+1,j]
                beta[t,i] = sum / scale[t+1]
    
    # Viterbi算法
    # 输入：A，B，pi,O 输出P(O|lambda)最大时Poptimal的路径I
    def viterbi(self,O):
        T = len(O)
        # 初始化
        delta = np.zeros((T,self.N),np.float)  
        phi = np.zeros((T,self.N),np.float)  
        I = np.zeros(T)
        for i in range(self.N):  
            delta[0,i] = self.pi[i]*self.B[i,O[0]]  
            phi[0,i] = 0
        # 递推
        for t in range(1,T):  
            for i in range(self.N):                                  
                delta[t,i] = self.B[i,O[t]]*np.array([delta[t-1,j]*self.A[j,i]  for j in range(self.N)]).max()
                phi[t,i] = np.array([delta[t-1,j]*self.A[j,i]  for j in range(self.N)]).argmax()
        # 终结
        prob = delta[T-1,:].max()  
        I[T-1] = delta[T-1,:].argmax()
        # 状态序列求取   
        for t in range(T-2,-1,-1): 
            I[t] = phi[t+1,I[t+1]]
        return I,prob
    
    # 计算gamma : 时刻t时马尔可夫链处于状态Si的概率    
    def ComputeGamma(self, T, alpha, beta, gamma):
        for t in range(T):
            denominator = 0.0
            for j in range(self.N):
                gamma[t,j] = alpha[t,j]*beta[t,j]
                denominator += gamma[t,j]
            for i in range(self.N):
                gamma[t,i] = gamma[t,i]/denominator
    
    # 计算sai(i,j) 为给定训练序列O和模型lambda时：
    # 时刻t是马尔可夫链处于Si状态，二时刻t+1处于Sj状态的概率
    def ComputeXi(self,T,O,alpha,beta,gamma,xi):
        for t in range(T-1):
            sum = 0.0
            for i in range(self.N):
                for j in range(self.N):
                    xi[t,i,j] = alpha[t,i]*beta[t+1,j]*self.A[i,j]*self.B[j,O[t+1]]
                    sum += xi[t,i,j]
            for i in range(self.N):
                for j in range(self.N):
                    xi[t,i,j] /= sum
                    
    # Baum-Welch算法
    # 输入 L个观察序列O，初始模型：HMM={A,B,pi,N,M}
    def BaumWelch(self,L,T,O,alpha,beta,gamma):
        print ("run BaumWelch")
        DELTA = 0.01 ; round = 0 ; flag = 1 ; probf = [0.0]
        delta = 0.0 ; deltaprev = 0.0 ; probprev = 0.0 ; ratio = 0.0 ; deltaprev = 10e-70
        
        xi = np.zeros((T,self.N,self.N))
        pi = np.zeros((T),np.float)
        denominatorA = np.zeros((self.N),np.float)
        denominatorB = np.zeros((self.N),np.float)
        numeratorA = np.zeros((self.N,self.N),np.float)
        numeratorB = np.zeros((self.N,self.M),np.float)
        scale = np.zeros((T),np.float)
        
        while True :
            probf[0] = 0
            # E - step
            for l in range(L):
                self.ForwardWithScale(T,O[l],alpha,scale,probf)
                self.BackwardWithScale(T,O[l],beta,scale)
                self.ComputeGamma(T,alpha,beta,gamma)
                self.ComputeXi(T,O[l],alpha,beta,gamma,xi)
                for i in range(self.N):
                    pi[i] += gamma[0,i]
                    for t in range(T-1): 
                        denominatorA[i] += gamma[t,i]
                        denominatorB[i] += gamma[t,i]
                    denominatorB[i] += gamma[T-1,i]
                    
                    for j in range(self.N):
                        for t in range(T-1):
                            numeratorA[i,j] += xi[t,i,j]
                    for k in range(self.M):
                        for t in range(T):
                            if O[l][t] == k:
                                numeratorB[i,k] += gamma[t,i]
                            
            # M - step
            # 重估状态转移矩阵 和 观察概率矩阵
            for i in range(self.N):
                self.pi[i] = 0.001/self.N + 0.999*pi[i]/L
                for j in range(self.N):
                    self.A[i,j] = 0.001/self.N + 0.999*numeratorA[i,j]/denominatorA[i]
                    numeratorA[i,j] = 0.0
                
                for k in range(self.M):
                    self.B[i,k] = 0.001/self.M + 0.999*numeratorB[i,k]/denominatorB[i]
                    numeratorB[i,k] = 0.0
                
                pi[i]=denominatorA[i]=denominatorB[i]=0.0;
            
            if flag == 1:
                flag = 0
                probprev = probf[0]
                ratio = 1
                continue
            
            delta = probf[0] - probprev
            ratio = delta / deltaprev
            probprev = probf[0]
            deltaprev = delta
            round += 1
            
            if ratio <= DELTA :
                print ("num iteration ",round)
                break
         

if __name__ == "__main__":
    print ("python my HMM")
  
    A = [[0.8125,0.1875],[0.2,0.8]]
    B = [[0.875,0.125],[0.25,0.75]]
    pi = [0.5,0.5]
    hmm = HMM(A,B,pi)
    
    O = [[1,0,0,1,1,0,0,0,0],
     [1,1,0,1,0,0,1,1,0],
     [0,0,1,1,0,0,1,1,1]]
    L = len(O)
    T = len(O[0])  # T等于最长序列的长度就好了
    alpha = np.zeros((T,hmm.N),np.float)
    beta = np.zeros((T,hmm.N),np.float)
    gamma = np.zeros((T,hmm.N),np.float)
    hmm.BaumWelch(L,T,O,alpha,beta,gamma)
    
    hmm.printhmm()

python my HMM
BaumWelch
num iteration  2140
HMM content: N = 2 ,M = 2
hmm.A  [ 0.59077362  0.40922638]  hmm.B  [ 0.9636068  0.0363932]
       [ 0.50971332  0.49028668]         [ 0.02315302  0.97684698]
hmm.pi [ 0.33220246  0.66779754]


## complete implementation 2

In [45]:
import numpy as np

DELTA = 0.001

class HMM:


    def __init__(self, pi, A, B):
        self.pi = pi
        self.A = A
        self.B = B
        self.M = B.shape[1]
        self.N = A.shape[0]
        
    def forward(self,obs):
        T = len(obs)
        N = self.N
        
        alpha = np.zeros([N,T])
        alpha[:,0] = self.pi[:] * self.B[:,obs[0]-1]                                                                                                      
    
        for t in xrange(1,T):
            for n in xrange(0,N):
                alpha[n,t] = np.sum(alpha[:,t-1] * self.A[:,n]) * self.B[n,obs[t]-1]
                     
        prob = np.sum(alpha[:,T-1])
        return prob, alpha
        
    def forward_with_scale(self, obs):
        """see scaling chapter in "A tutorial on hidden Markov models and 
        selected applications in speech recognition." 
        """
        T = len(obs)
        N = self.N
        alpha = np.zeros([N,T])
        scale = np.zeros(T)

        alpha[:,0] = self.pi[:] * self.B[:,obs[0]-1]
        scale[0] = np.sum(alpha[:,0])
        alpha[:,0] /= scale[0]

        for t in xrange(1,T):
            for n in xrange(0,N):
                alpha[n,t] = np.sum(alpha[:,t-1] * self.A[:,n]) * self.B[n,obs[t]-1]
            scale[t] = np.sum(alpha[:,t])
            alpha[:,t] /= scale[t]

        logprob = np.sum(np.log(scale[:]))
        return logprob, alpha, scale    
        
    def backward(self, obs):
        T = len(obs)
        N = self.N
        beta = np.zeros([N,T])
        
        beta[:,T-1] = 1
        for t in reversed(xrange(0,T-1)):
            for n in xrange(0,N):
                beta[n,t] = np.sum(self.B[:,obs[t+1]-1] * self.A[n,:] * beta[:,t+1])
                
        prob = np.sum(beta[:,0])
        return prob, beta

    def backward_with_scale(self, obs, scale):
        T = len(obs)
        N = self.N
        beta = np.zeros([N,T])

        beta[:,T-1] = 1 / scale[T-1]
        for t in reversed(xrange(0,T-1)):
            for n in xrange(0,N):
                beta[n,t] = np.sum(self.B[:,obs[t+1]-1] * self.A[n,:] * beta[:,t+1])
                beta[n,t] /= scale[t]
        
        return beta

    def viterbi(self, obs):
        T = len(obs)
        N = self.N
        psi = np.zeros([N,T]) # reverse pointer
        delta = np.zeros([N,T])
        q = np.zeros(T)
        temp = np.zeros(N)        
        
        delta[:,0] = self.pi[:] * self.B[:,obs[0]-1]    
        
        for t in xrange(1,T):
            for n in xrange(0,N):
                temp = delta[:,t-1] * self.A[:,n]    
                max_ind = argmax(temp)
                psi[n,t] = max_ind
                delta[n,t] = self.B[n,obs[t]-1] * temp[max_ind]

        max_ind = argmax(delta[:,T-1])
        q[T-1] = max_ind
        prob = delta[:,T-1][max_ind]

        for t in reversed(xrange(1,T-1)):
            q[t] = psi[q[t+1],t+1]    
            
        return prob, q, delta    
        
    def viterbi_log(self, obs):
        
        T = len(obs)
        N = self.N
        psi = np.zeros([N,T])
        delta = np.zeros([N,T])
        pi = np.zeros(self.pi.shape)
        A = np.zeros(self.A.shape)
        biot = np.zeros([N,T])

        pi = np.log(self.pi)        
        A = np.log(self.A)
        biot = np.log(self.B[:,obs[:]-1])

        delta[:,0] = pi[:] + biot[:,0]

        for t in xrange(1,T):
            for n in xrange(0,N):
                temp = delta[:,t-1] + self.A[:,n]    
                max_ind = argmax(temp)
                psi[n,t] = max_ind
                delta[n,t] = temp[max_ind] + biot[n,t]   

        max_ind = argmax(delta[:,T-1])
        q[T-1] = max_ind            
        logprob = delta[max_ind,T-1]      
                
        for t in reversed(xrange(1,T-1)):
            q[t] = psi[q[t+1],t+1]    

        return logprob, q, delta

    def baum_welch(self, obs):
        T = len(obs)
        M = self.M
        N = self.N        
        alpha = np.zeros([N,T])
        beta = np.zeros([N,T])
        scale = np.zeros(T)
        gamma = np.zeros([N,T])
        xi = np.zeros([N,N,T-1])
    
        # caculate initial parameters
        logprobprev, alpha, scale = self.forward_with_scale(obs)
        beta = self.backward_with_scale(obs, scale)            
        gamma = self.compute_gamma(alpha, beta)    
        xi = self.compute_xi(obs, alpha, beta)    
        logprobinit = logprobprev        
        
        # start interative 
        while True:
            # E-step
            self.pi = 0.001 + 0.999*gamma[:,0]
            for i in xrange(N):
                denominator = np.sum(gamma[i,0:T-1])
                for j in xrange(N): 
                    numerator = np.sum(xi[i,j,0:T-1])
                    self.A[i,j] = numerator / denominator
                                   
            self.A = 0.001 + 0.999*self.A
            for j in xrange(0,N):
                denominator = np.sum(gamma[j,:])
                for k in xrange(0,M):
                    numerator = 0.0
                    for t in xrange(0,T):
                        if obs[t]-1 == k:
                            numerator += gamma[j,t]
                    self.B[j,k] = numerator / denominator
            self.B = 0.001 + 0.999*self.B

            # M-step
            logprobcur, alpha, scale = self.forward_with_scale(obs)
            beta = self.backward_with_scale(obs, scale)            
            gamma = self.compute_gamma(alpha, beta)    
            xi = self.compute_xi(obs, alpha, beta)    

            delta = logprobcur - logprobprev
            logprobprev = logprobcur
            # print "delta is ", delta
            if delta <= DELTA:
                break     
                
        logprobfinal = logprobcur
        return logprobinit, logprobfinal                
            
    def compute_gamma(self, alpha, beta):
        gamma = np.zeros(alpha.shape)
        gamma = alpha[:,:] * beta[:,:]
        gamma = gamma / np.sum(gamma,0)
        return gamma
            
    def compute_xi(self, obs, alpha, beta):
        T = len(obs)
        N = self.N
        xi = np.zeros((N, N, T-1))
            
        for t in xrange(0,T-1):        
            for i in xrange(0,N):
                for j in xrange(0,N):
                    xi[i,j,t] = alpha[i,t] * self.A[i,j] * \
                                self.B[j,obs[t+1]-1] * beta[j,t+1]
            xi[:,:,t] /= np.sum(np.sum(xi[:,:,t],1),0)    
        return xi

def read_hmm(hmmfile):
    fhmm = open(hmmfile,'r') 

    M = int(fhmm.readline().split(' ')[1])
    N = int(fhmm.readline().split(' ')[1]) 
    
    A = np.array([])
    fhmm.readline()
    for i in xrange(N):
        line = fhmm.readline()
        if i == 0:
            A = np.array(map(float,line.split(',')))
        else:
            A = np.vstack((A,map(float,line.split(','))))
        
    B = np.array([])
    fhmm.readline()
    for i in xrange(N):
        line = fhmm.readline()
        if i == 0:
            B = np.array(map(float,line.split(',')))
        else:
            B = np.vstack((B,map(float,line.split(','))))
    
    fhmm.readline()
    line = fhmm.readline()
    pi = np.array(map(float,line.split(',')))
    
    fhmm.close()
    return M, N, pi, A, B 
    
def read_sequence(seqfile):
    fseq = open(seqfile,'r') 
    
    T = int(fseq.readline().split(' ')[1])
    line = fseq.readline()
    obs = np.array(map(int,line.split(',')))
    
    fseq.close()
    return T, obs

## complete implementation 3
参考自：https://github.com/hankcs/hidden-markov-model

In [66]:
import numpy as np

class HMM:
    """
    Order 1 Hidden Markov Model
    Attributes
    ----------
    A : numpy.ndarray
        State transition probability matrix
    B: numpy.ndarray
        Output emission probability matrix with shape(N, number of output types)
    pi: numpy.ndarray
        Initial state probablity vector
    Common Variables
    ----------------
    obs_seq : list of int
        list of observations (represented as ints corresponding to output
        indexes in B) in order of appearance
    T : int
        number of observations in an observation sequence
    N : int
        number of states
    """

    def __init__(self, A, B, pi):
        self.A = A
        self.B = B
        self.pi = pi

    def _forward(self, obs_seq):
        N = self.A.shape[0]
        T = len(obs_seq)

        F = np.zeros((N,T))
        F[:,0] = self.pi * self.B[:, obs_seq[0]]

        for t in range(1, T):
            for n in range(N):
                F[n,t] = np.dot(F[:,t-1], (self.A[:,n])) * self.B[n, obs_seq[t]]

        return F

    def _backward(self, obs_seq):
        N = self.A.shape[0]
        T = len(obs_seq)

        X = np.zeros((N,T))
        X[:,-1:] = 1

        for t in reversed(range(T-1)):
            for n in range(N):
                X[n,t] = np.sum(X[:,t+1] * self.A[n,:] * self.B[:, obs_seq[t+1]])

        return X

    def observation_prob(self, obs_seq):
        """ P( entire observation sequence | A, B, pi ) """
        return np.sum(self._forward(obs_seq)[:,-1])

    def state_path(self, obs_seq):
        """
        Returns
        -------
        V[last_state, -1] : float
            Probability of the optimal state path
        path : list(int)
            Optimal state path for the observation sequence
        """
        V, prev = self.viterbi(obs_seq)

        # Build state path with greatest probability
        last_state = np.argmax(V[:,-1])
        path = list(self.build_viterbi_path(prev, last_state))

        return V[last_state,-1], reversed(path)

    def viterbi(self, obs_seq):
        """
        Returns
        -------
        V : numpy.ndarray
            V [s][t] = Maximum probability of an observation sequence ending
                       at time 't' with final state 's'
        prev : numpy.ndarray
            Contains a pointer to the previous state at t-1 that maximizes
            V[state][t]
        """
        N = self.A.shape[0]
        T = len(obs_seq)
        prev = np.zeros((T - 1, N), dtype=int)

        # DP matrix containing max likelihood of state at a given time
        V = np.zeros((N, T))
        V[:,0] = self.pi * self.B[:,obs_seq[0]]

        for t in range(1, T):
            for n in range(N):
                seq_probs = V[:,t-1] * self.A[:,n] * self.B[n, obs_seq[t]]
                prev[t-1,n] = np.argmax(seq_probs)
                V[n,t] = np.max(seq_probs)

        return V, prev

    def build_viterbi_path(self, prev, last_state):
        """Returns a state path ending in last_state in reverse order."""
        T = len(prev)
        yield(last_state)
        for i in range(T-1, -1, -1):
            yield(prev[i, last_state])
            last_state = prev[i, last_state]

    def simulate(self, T):

        def draw_from(probs):
            return np.where(np.random.multinomial(1,probs) == 1)[0][0]

        observations = np.zeros(T, dtype=int)
        states = np.zeros(T, dtype=int)
        states[0] = draw_from(self.pi)
        observations[0] = draw_from(self.B[states[0],:])
        for t in range(1, T):
            states[t] = draw_from(self.A[states[t-1],:])
            observations[t] = draw_from(self.B[states[t],:])
        return observations,states

    def baum_welch_train(self, observations, criterion=0.05):
        n_states = self.A.shape[0]
        n_samples = len(observations)

        done = False
        while not done:
            # alpha_t(i) = P(O_1 O_2 ... O_t, q_t = S_i | hmm)
            # Initialize alpha
            alpha = self._forward(observations)

            # beta_t(i) = P(O_t+1 O_t+2 ... O_T | q_t = S_i , hmm)
            # Initialize beta
            beta = self._backward(observations)

            xi = np.zeros((n_states,n_states,n_samples-1))
            for t in range(n_samples-1):
                denom = np.dot(np.dot(alpha[:,t].T, self.A) * self.B[:,observations[t+1]].T, beta[:,t+1])
                for i in range(n_states):
                    numer = alpha[i,t] * self.A[i,:] * self.B[:,observations[t+1]].T * beta[:,t+1].T
                    xi[i,:,t] = numer / denom

            # gamma_t(i) = P(q_t = S_i | O, hmm)
            gamma = np.squeeze(np.sum(xi,axis=1))
            # Need final gamma element for new B
            prod =  (alpha[:,n_samples-1] * beta[:,n_samples-1]).reshape((-1,1))
            gamma = np.hstack((gamma,  prod / np.sum(prod))) #append one more to gamma!!!

            newpi = gamma[:,0]
            newA = np.sum(xi,2) / np.sum(gamma[:,:-1],axis=1).reshape((-1,1))
            newB = np.copy(self.B)

            num_levels = self.B.shape[1]
            sumgamma = np.sum(gamma,axis=1)
            for lev in range(num_levels):
                mask = observations == lev
                newB[:,lev] = np.sum(gamma[:,mask],axis=1) / sumgamma

            if np.max(abs(self.pi - newpi)) < criterion and \
                            np.max(abs(self.A - newA)) < criterion and \
                            np.max(abs(self.B - newB)) < criterion:
                done = 1

            self.A[:],self.B[:],self.pi[:] = newA,newB,newpi
            #print("update") 
            
states = ('Healthy', 'Fever')

observations = ('normal', 'cold', 'dizzy')

start_probability = {'Healthy': 0.6, 'Fever': 0.4}

transition_probability = {
    'Healthy': {'Healthy': 0.7, 'Fever': 0.3},
    'Fever': {'Healthy': 0.4, 'Fever': 0.6},
}

emission_probability = {
    'Healthy': {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
    'Fever': {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6},
}

##############转换数据类型###########
def generate_index_map(lables):
    index_label = {}
    label_index = {}
    i = 0
    for l in lables:
        index_label[i] = l
        label_index[l] = i
        i += 1
    return label_index, index_label


states_label_index, states_index_label = generate_index_map(states)
observations_label_index, observations_index_label = generate_index_map(observations)


def convert_observations_to_index(observations, label_index):
    list = []
    for o in observations:
        list.append(label_index[o])
    return list


def convert_map_to_vector(map, label_index):
    v = np.empty(len(map), dtype=float)
    for e in map:
        v[label_index[e]] = map[e]
    return v


def convert_map_to_matrix(map, label_index1, label_index2):
    m = np.empty((len(label_index1), len(label_index2)), dtype=float)
    for line in map:
        for col in map[line]:
            m[label_index1[line]][label_index2[col]] = map[line][col]
    return m


A = convert_map_to_matrix(transition_probability, states_label_index, states_label_index)
# print A
B = convert_map_to_matrix(emission_probability, states_label_index, observations_label_index)
# print B
observations_index = convert_observations_to_index(observations, observations_label_index)
pi = convert_map_to_vector(start_probability, states_label_index)
# print pi
##############转换数据类型###########
print('--------------------------viterbi test-----------------------------------')
h = HMM(A, B, pi)
V, p = h.viterbi(observations_index)
print(" " * 7, " ".join(("%10s" % observations_index_label[i]) for i in observations_index))
for s in range(0, 2):
    print ("%7s: " % states_index_label[s] + " ".join("%10s" % ("%f" % v) for v in V[s]))
print('\nThe most possible states and probability are:')
p, ss = h.state_path(observations_index)
for s in ss:
    print(states_index_label[s],end=' ')
print(p)

simulength = 100
# run a baum_welch_train
observations_data, states_data = h.simulate(simulength)
# print observations_data
# print states_data
guess = HMM(np.array([[0.5, 0.5],
                          [0.5, 0.5]]),
                np.array([[0.3, 0.3, 0.3],
                          [0.3, 0.3, 0.3]]),
                np.array([0.5, 0.5])
                )
guess.baum_welch_train(observations_data)
states_out = np.array(list(guess.state_path(observations_data)[1]))
print('\n\n-----------------------baum_welch_train test-------------------------------')
print(states_out) #这个变量输出来竟然都是0，应该是程序有bug
print(states_data)

compare=states_out-states_data
#输出准确率
print('Accuracy： %.9f' % (len(compare[compare==0]) / simulength))

--------------------------viterbi test-----------------------------------
            normal       cold      dizzy
Healthy:   0.300000   0.084000   0.005880
  Fever:   0.040000   0.027000   0.015120

The most possible states and probability are:
Healthy Healthy Fever 0.01512
update
update


-----------------------baum_welch_train test-------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0 0 0
 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0]
Accuracy： 0.610000000
